In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import EURUSD from backtesting.py package

# df = pd.read_table('/Users/newuser/Projects/robust-algo-trader/data/EURUSD_H1_200702210000_202304242100.tsv')
df = pd.read_table('/Users/newuser/Projects/robust-algo-trader/data/EURUSD_H1_202302010000_202304242100.tsv')
# remove the following columns <TICKVOL>, <VOL> and <SPREAD>
df = df.drop(['<TICKVOL>', '<VOL>', '<SPREAD>'], axis=1)
# rename the columns
df = df.rename(columns={'<DATE>': 'Date', 
                                '<TIME>': 'Time', 
                                '<OPEN>': 'Open', 
                                '<HIGH>': 'High', 
                                '<LOW>': 'Low', 
                                '<CLOSE>': 'Close'})
# combine the date and time columns
df['Date_Time'] = df['Date'] + ' ' + df['Time']
# convert the date_time column to datetime
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format='%Y%m%d %H:%M:%S.%f')
# remove the date and time columns
df = df.drop(['Date', 'Time'], axis=1)
df.index = df['Date_Time']

# define the time period for the opening range
period = 15 # minutes

# calculate the high and low of the opening range
df["OR_high"] = df.groupby(df.index.date)["High"].transform(lambda x: x.iloc[:period].max())
df["OR_low"] = df.groupby(df.index.date)["Low"].transform(lambda x: x.iloc[:period].min())

# define the breakout level as a percentage above or below the opening range high or low
breakout_level = 0.005 # 0.5%

# create buy and sell signals based on the breakout level and the current price
df["Buy"] = df.apply(lambda x: x["Close"] > x["OR_high"] * (1 + breakout_level), axis=1).astype(int)
df["Sell"] = df.apply(lambda x: x["Close"] < x["OR_low"] * (1 - breakout_level), axis=1).astype(int)

# optionally, add stop loss and take profit levels based on a multiple of the opening range size or a fixed amount
stop_loss_multiple = 2 # stop loss is 2x of the opening range size
take_profit_multiple = 3 # take profit is 2x of the opening range size

df["OR_size"] = df["OR_high"] - df["OR_low"]
df["Stop_loss"] = np.where(df["Buy"] == 1, df["OR_low"] - df["OR_size"] * stop_loss_multiple,
                           np.where(df["Sell"] == 1, df["OR_high"] + df["OR_size"] * stop_loss_multiple, np.nan))
df["Take_profit"] = np.where(df["Buy"] == 1, df["OR_high"] + df["OR_size"] * take_profit_multiple,
                             np.where(df["Sell"] == 1, df["OR_low"] - df["OR_size"] * take_profit_multiple, np.nan))

df = df.reset_index(drop=True)
# initialize an empty list to store the signals
signals = []
# initialize a variable to track the position status
position = 0
non_na_take_profit = np.nan
non_na_stop_loss = np.nan

# loop through the rows of the dataframe
for i, row in df.iterrows():
    current_take_profit = row['Take_profit']
    current_stop_loss = row['Stop_loss']
    
    prev_take_profit = df['Take_profit'].iloc[i-1] if i > 0 else np.nan
    next_take_profit = df['Take_profit'].iloc[i+1] if i < len(df)-1 else np.nan
    
    if ~np.isnan(current_take_profit):
        non_na_take_profit = current_take_profit
        
    if ~np.isnan(non_na_stop_loss):
        non_na_stop_loss = current_stop_loss
    
    
    if row['Buy'] == 1 and prev_take_profit != current_take_profit:
        position = 1
        signals.append(1)
    # if there is a long position and there is a sell signal or the price hits the stop loss or take profit level, sell and set position to -1
    elif row['Sell'] == 1 and prev_take_profit != current_take_profit:
        position = -1
        signals.append(-1)
    # if there is a short position and there is a buy signal or the price hits the stop loss or take profit level, close the deal and set position to -2
    else:
        current_close_val = row['Close']        
        if position == 1 and (current_close_val <= non_na_stop_loss or current_close_val >= non_na_take_profit):
            signals.append(-2)
            position = -2
        elif position == -1 and (current_close_val >= non_na_stop_loss or current_close_val <= non_na_take_profit):
            signals.append(-2) 
            position = -2
        else:
            signals.append(0)       
            # position = 0


# add a new column for signals to the dataframe
df['Signal'] = signals
df.index = df['Date_Time']

In [23]:
df

,Open,High,Low,Close,Date_Time,OR_high,OR_low,Buy,Sell,OR_size,Stop_loss,Take_profit,Signal
Date_Time,,,,,,,,,,,,,
2023-02-01 00:00:00,1.08607,1.08639,1.08577,1.08606,2023-02-01 00:00:00,1.08979,1.08523,0,0,0.00456,NaN,NaN,0
2023-02-01 01:00:00,1.08606,1.08654,1.08591,1.08604,2023-02-01 01:00:00,1.08979,1.08523,0,0,0.00456,NaN,NaN,0
2023-02-01 02:00:00,1.08607,1.08642,1.08585,1.08624,2023-02-01 02:00:00,1.08979,1.08523,0,0,0.00456,NaN,NaN,0
2023-02-01 03:00:00,1.08624,1.08637,1.08571,1.08573,2023-02-01 03:00:00,1.08979,1.08523,0,0,0.00456,NaN,NaN,0
2023-02-01 04:00:00,1.08572,1.08572,1.08523,1.08538,2023-02-01 04:00:00,1.08979,1.08523,0,0,0.00456,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-24 17:00:00,1.10237,1.10370,1.10207,1.10294,2023-04-24 17:00:00,1.10205,1.09659,0,0,0.00546,NaN,NaN,0
2023-04-24 18:00:00,1.10294,1.10323,1.10206,1.10260,2023-04-24 18:00:00,1.10205,1.09659,0,0,0.00546,NaN,NaN,0
2023-04-24 19:00:00,1.10260,1.10342,1.10249,1.10340,2023-04-24 19:00:00,1.10205,1.09659,0,0,0.00546,NaN,NaN,0


In [26]:


from backtesting import Backtest
from backtesting import Strategy

class CustomStrategy(Strategy):
    def init(self):
        pass
    
    def next(self):
        current_signal = self.data.Signal[-1]
        if current_signal == 1:
            self.buy()
        elif current_signal == -1:
            self.sell()
        elif current_signal == -2:
            self.position.close() 
        

bt = Backtest(df, CustomStrategy, commission=.002,
              cash=2000,
              exclusive_orders=True)
stats = bt.run()

bt.plot()
print(stats)
stats._strategy
bt.plot(plot_volume=False, plot_pl=False)
stats.tail()
print(stats['_trades']) 

# print the dataframe
# print(df)
        
'''
# plot the price chart with the opening range and the signals
plt.figure(figsize=(10, 6))
plt.plot(df["Close"], color="blue", label="Price")
plt.plot(df["OR_high"], color="green", linestyle="--", label="Opening Range High")
plt.plot(df["OR_low"], color="red", linestyle="--", label="Opening Range Low")
plt.plot(df[df["Buy"] == 1]["Close"], color="green", marker="^", markersize=10, label="Buy")
plt.plot(df[df["Sell"] == 1]["Close"], color="red", marker="v", markersize=10, label="Sell")
plt.plot(df[df["Buy"] == 1]["Stop_loss"], color="green", marker="_", markersize=10, label="Stop Loss")
plt.plot(df[df["Sell"] == 1]["Stop_loss"], color="red", marker="_", markersize=10, label="Stop Loss")

# plot the price chart with the opening range and the signals
plt.figure(figsize=(10, 6))
plt.plot(df["Close"], color="blue", label="Price")
plt.plot(df["OR_high"], color="green", linestyle="--", label="Opening Range High")
plt.plot(df["OR_low"], color="red", linestyle="--", label="Opening Range Low")
plt.plot(df[df["Buy"] == 1]["Close"], color="green", marker="^", markersize=10, label="Buy")
plt.plot(df[df["Sell"] == 1]["Close"], color="red", marker="v", markersize=10, label="Sell")
plt.plot(df[df["Buy"] == 1]["Stop_loss"], color="green", marker="_", markersize=10, label="Stop Loss")
plt.plot(df[df["Sell"] == 1]["Stop_loss"], color="red", marker="_", markersize=10, label="Stop Loss")
plt.plot(df[df["Buy"] == 1]["Take_profit"], color="green", marker="+", markersize=10, label="Take Profit")
plt.plot(df[df["Sell"] == 1]["Take_profit"], color="red", marker="+", markersize=10, label="Take Profit")
plt.legend()
plt.show()
'''

Start                     2023-02-01 00:00:00
End                       2023-04-24 21:00:00
Duration                     82 days 21:00:00
Exposure Time [%]                    28.68272
Equity Final [$]                  2004.549784
Equity Peak [$]                   2012.533772
Return [%]                           0.227489
Buy & Hold Return [%]                1.712613
Return (Ann.) [%]                    1.425631
Volatility (Ann.) [%]                6.243572
Sharpe Ratio                         0.228336
Sortino Ratio                        0.308964
Calmar Ratio                         0.526597
Max. Drawdown [%]                   -2.707252
Avg. Drawdown [%]                   -0.588891
Max. Drawdown Duration       47 days 16:00:00
Avg. Drawdown Duration        6 days 15:00:00
# Trades                                   10
Win Rate [%]                             60.0
Best Trade [%]                       0.835379
Worst Trade [%]                     -0.971872
Avg. Trade [%]                    

'\n# plot the price chart with the opening range and the signals\nplt.figure(figsize=(10, 6))\nplt.plot(df["Close"], color="blue", label="Price")\nplt.plot(df["OR_high"], color="green", linestyle="--", label="Opening Range High")\nplt.plot(df["OR_low"], color="red", linestyle="--", label="Opening Range Low")\nplt.plot(df[df["Buy"] == 1]["Close"], color="green", marker="^", markersize=10, label="Buy")\nplt.plot(df[df["Sell"] == 1]["Close"], color="red", marker="v", markersize=10, label="Sell")\nplt.plot(df[df["Buy"] == 1]["Stop_loss"], color="green", marker="_", markersize=10, label="Stop Loss")\nplt.plot(df[df["Sell"] == 1]["Stop_loss"], color="red", marker="_", markersize=10, label="Stop Loss")\n\n# plot the price chart with the opening range and the signals\nplt.figure(figsize=(10, 6))\nplt.plot(df["Close"], color="blue", label="Price")\nplt.plot(df["OR_high"], color="green", linestyle="--", label="Opening Range High")\nplt.plot(df["OR_low"], color="red", linestyle="--", label="Openi